# T81-558: Applications of Deep Neural Networks
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), School of Engineering and Applied Science, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

**Module 6 Assignment: Classification Neural Network**

**Student Name: Jie Miao**

# Assignment Instructions

For this assignment you will use the **cls-37-spring-2019.csv** dataset.  This is a dataset that I generated specifically for this semester.  You can find the CSV file on my data site, at this location: [reg-33-spring-2019.csv](http://data.heatonresearch.com/data/t81-558/datasets/cls-37-spring-2019.csv).

You will train 5 neural networks, one for each fold of a 5-fold cross validation and return the out of sample predictions.  You will submit these perdictions to the **submit** function.  See [Assignment #1](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/assignments/assignment_yourname_class1.ipynb) for details on how to submit an assignment or check that one was submitted.

Complete the following tasks:

* Normalize all numerics to zscores and all text/categoricals to dummies.  Normalize the *target* to an index (use *encode_text_index*).
* Your target (y) is the filed named *target*
* If you find any missing values (NA's), replace them with the median values for that column.
* Use a 5-fold cross validation and return out of sample predictions.  
* Your submission should contain the id (column name *id*), your prediction (column name *pred*), the expected value (from the **cls-32-spring-2018.csv** dataset. Note that both *pred* and *y* should be in index form (i.e. 1,2,3,4 etc, not the text form, TA-0, TA-1, etc).  The accuarcy I got for this one was in the 90's.
* You might get warnings about the means of your columns differing from mine.  Do not worry about small differences.  
* Your submitted dataframe will have these columns: id, y, pred.


# Helpful Functions

You will see these at the top of every module and assignment.  These are simply a set of reusable functions that we will make use of.  Each of them will be explained as the semester progresses.  They are explained in greater detail as the course progresses.  Class 4 contains a complete overview of these functions.

In [8]:
import base64
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
from sklearn import preprocessing


# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = f"{name}-{x}"
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


# Encode text values to a single dummy variable.  The new columns (which do not replace the old) will have a 1
# at every location where the original column (name) matches each of the target_values.  One column is added for
# each target value.
def encode_text_single_dummy(df, name, target_values):
    for tv in target_values:
        l = list(df[name].astype(str))
        l = [1 if str(x) == str(tv) else 0 for x in l]
        name2 = f"{name}-{tv}"
        df[name2] = l


# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(
        target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    # Regression
    return df[result].values.astype(np.float32), df[[target]].values.astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return f"{h}:{m:>02}:{s:>05.2f}"


# Regression chart.
def chart_regression(pred, y, sort=True):
    t = pd.DataFrame({'pred': pred, 'y': y.flatten()})
    if sort:
        t.sort_values(by=['y'], inplace=True)
    plt.plot(t['y'].tolist(), label='expected')
    plt.plot(t['pred'].tolist(), label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean())
                          >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
        * (normalized_high - normalized_low) + normalized_low


# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - Pandas dataframe output.
# key - Your student key that was emailed to you.
# no - The assignment class number, should be 1 through 1.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.  
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.
def submit(data,key,no,source_file=None):
    if source_file is None and '__file__' not in globals(): raise Exception('Must specify a filename when a Jupyter notebook.')
    if source_file is None: source_file = __file__
    suffix = '_class{}'.format(no)
    if suffix not in source_file: raise Exception('{} must be part of the filename.'.format(suffix))
    with open(source_file, "rb") as image_file:
        encoded_python = base64.b64encode(image_file.read()).decode('ascii')
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb','.py']: raise Exception("Source file is {} must be .py or .ipynb".format(ext))
    r = requests.post("https://api.heatonresearch.com/assignment-submit",
        headers={'x-api-key':key}, json={'csv':base64.b64encode(data.to_csv(index=False).encode('ascii')).decode("ascii"),
        'assignment': no, 'ext':ext, 'py':encoded_python})
    if r.status_code == 200:
        print("Success: {}".format(r.text))
    else: print("Failure: {}".format(r.text))

# Assignment #6 Sample Code

The following code provides a starting point for this assignment.

In [67]:
import os
import pandas as pd
from scipy.stats import zscore
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
import pandas as pd
import io
import requests
import numpy as np
from sklearn import metrics
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
# This is your student key that I emailed to you at the beginnning of the semester.
key = "qqeAIdN7kdaHWUmLYRyM5aHJxb8Yt5Ssw0qi11wb"  # This is an example key and will not work.

# You must also identify your source file.  (modify for your local setup)
# file='/resources/t81_558_deep_learning/assignment_yourname_class1.ipynb'  # IBM Data Science Workbench
# file='C:\\Users\\jeffh\\projects\\t81_558_deep_learning\\t81_558_class1_intro_python.ipynb'  # Windows
file='/Users/jie/Desktop/t81_558_deep_learning-master/assignments/assignment_JieMiao_class6.ipynb'  # Mac/Linux
# file = "C:\\Users\\jeffh\\Dropbox\\school\\teaching\\wustl\\classes\\T81_558_deep_learning\\solutions\\assignment_solution_class6.ipynb"

# Begin assignment
path ="/Users/jie/Desktop"

filename_read = os.path.join(path,"cls-37-spring-2019.csv")
df = pd.read_csv(filename_read, na_values=['NA', '?'])

# Add assignment code here

#Normalize all numerics to zscores and all text/categoricals to dummies. 
#Normalize the target to an index (use encode_text_index).
float_col = df.select_dtypes(['float64']).columns.tolist()
int_col = df.select_dtypes(['int64']).columns.tolist()
cols = float_col + int_col
cols.append('target')
all_cols = df.columns.tolist()
for col in cols:
    if  col != "target" and col != "id":
        encode_numeric_zscore(df, col, mean=None, sd=None)

for col in all_cols:
    if col not in cols and col != 'target':
        encode_text_dummy(df, col)

#Your target (y) is the filed named target
#If you find any missing values (NA's), replace them with the median values for that column.
for col in df.columns.tolist():
    missing_median(df, cols)
    
#Use a 5-fold cross validation and return out of sample predictions.
encode_text_index(df,"target")
x,y = to_xy(df,"target")
kf = KFold(5)
    
oos_y = []
oos_pred = []
fold = 0

for train, test in kf.split(x):
    fold+=1
    print("Fold #{}".format(fold))
        
    x_train = x[train]
    y_train = y[train]
    x_test = x[test]
    y_test = y[test]
    
    model = Sequential()
    model.add(Dense(50, input_dim=x.shape[1], activation='relu')) # Hidden 1
    model.add(Dense(25, activation='relu')) # Hidden 2
    model.add(Dense(y.shape[1],activation='softmax')) # Output
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=25, verbose=1, mode='auto')

    model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor],verbose=0,epochs=1000)
    
    pred = model.predict(x_test)
    
    oos_y.append(y_test)
    pred = np.argmax(pred,axis=1) # raw probabilities to chosen class (highest probability)
    oos_pred.append(pred)        

    # Measure this fold's accuracy
    y_compare = np.argmax(y_test,axis=1) # For accuracy calculation
    score = metrics.accuracy_score(y_compare, pred)
    print("Fold score (accuracy): {}".format(score))


# Build the oos prediction list and calculate the error.
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
oos_y_compare = np.argmax(oos_y,axis=1) # For accuracy calculation

score = metrics.accuracy_score(oos_y_compare, oos_pred)
print("Final score (accuracy): {}".format(score))


#Your submission should contain the id (column name id), your prediction (column name pred), the expected value (from the cls-32-spring-2018.csv dataset. Note that both pred and y should be in index form (i.e. 1,2,3,4 etc, not the text form, TA-0, TA-1, etc). The accuarcy I got for this one was in the 90's.
#You might get warnings about the means of your columns differing from mine. Do not worry about small differences.
#Your submitted dataframe will have these columns: id, y, pred.


Fold #1
Epoch 00026: early stopping
Fold score (accuracy): 0.19747695293546821
Fold #2
Epoch 00027: early stopping
Fold score (accuracy): 0.20184376516254246
Fold #3
Epoch 00026: early stopping
Fold score (accuracy): 0.20038816108685104
Fold #4
Epoch 00028: early stopping
Fold score (accuracy): 0.18825812712275594
Fold #5
Epoch 00026: early stopping
Fold score (accuracy): 0.19796215429403202
Final score (accuracy): 0.19718583212032995


In [107]:
df['y'] =pd.Series(oos_y.values.tolist())
df["pred"] = pd.Series(oos_pred.values.tolist())
df["pred"].head()
df['y'].head()
col_id = df['id']
col_pred = df['pred']
col_y = df['y']
oosDF = pd.concat([col_id, col_y, col_pred],axis=1 )
submit(source_file=file,data=oosDF,key=key,no=6)

Success: Submitted Assignment #6 for j.miao:
You have submitted this assignment 2 times. (this is fine)
No warnings on your data. You will probably do well, but no guarantee. :-)

